In [1]:
from time import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from ImagesloaderRAM import *
from Trainfunction import *
from Savemodel import *
from Mycudatransformation import *
from MynnModule import *
from PSNR import *

import torchvision.transforms as T
import torchvision.utils as tu

import numpy as np

%load_ext autoreload
%autoreload 2


dtype = torch.FloatTensor
gpu_dtype = torch.cuda.FloatTensor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')


print('using device:', device)

using device: cuda


## Train

In [3]:
# Loading the dataset of images


Data_preprocessing = T.Compose([T.RandomCrop(64),
                                T.ToTensor()])

trainset = ImageFolderRAM('./BSD200C', 0.25, Data_preprocessing,
                          Randomnoise=True, loader=RGB_loader, loadram='cpu')

trainloader = torch.utils.data.DataLoader(
    trainset, shuffle=True, batch_size=64, num_workers=0)

## Training from scratch

In [4]:
# Choose the model of neural networks architectures
Denoiser = SFD_C().to(device)

# Choose the loss function
loss = nn.L1Loss().to(device)

# Choose the Optimizer
optimizer = optim.Adam(Denoiser.parameters(), lr=0.001, betas=(0.9, 0.999),
                       eps=1e-08, weight_decay=0.00001, amsgrad=False)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.9, patience=5000, verbose=True, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)

In [5]:
def trainserver(model, loader_train, loss_fn, optimizer, scheduler, num_epochs=1, save_every=1, loss_every=10, filename='denoiser'):

    loss_history = []
    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        model.train()

        for t, (x, y) in enumerate(loader_train):
            x_var = Variable(x.to(device))
            y_var = Variable(y.to(device))

            frame = model(x_var)
            loss = loss_fn(frame, y_var)

            if (t + 1) % loss_every == 0:
                print('t = %d, loss = %.4f' % (t + 1, loss.data))
                loss_history.append(loss)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if (epoch+1) % save_every == 0:
            Save_modelloss(model, optimizer, loss_history,
                           filename+'%s ' % int(epoch+1))

        scheduler.step(loss.data)

In [ ]:
trainserver(Denoiser, trainloader, loss, optimizer, scheduler,
            num_epochs=500000, save_every=5000, loss_every=3, filename='SFD_C_NB')

Starting epoch 1 / 500000
t = 3, loss = 0.2435
Starting epoch 2 / 500000
t = 3, loss = 0.1186
Starting epoch 3 / 500000
t = 3, loss = 0.0934
Starting epoch 4 / 500000
t = 3, loss = 0.0834
Starting epoch 5 / 500000
t = 3, loss = 0.0787
Starting epoch 6 / 500000
t = 3, loss = 0.0769
Starting epoch 7 / 500000
t = 3, loss = 0.0812
Starting epoch 8 / 500000
t = 3, loss = 0.0753
Starting epoch 9 / 500000
t = 3, loss = 0.1216
Starting epoch 10 / 500000
t = 3, loss = 0.0717
Starting epoch 11 / 500000
t = 3, loss = 0.0705
Starting epoch 12 / 500000
t = 3, loss = 0.0647
Starting epoch 13 / 500000
t = 3, loss = 0.0596
Starting epoch 14 / 500000
t = 3, loss = 0.0768
Starting epoch 15 / 500000
t = 3, loss = 0.0621
Starting epoch 16 / 500000
t = 3, loss = 0.0772
Starting epoch 17 / 500000
t = 3, loss = 0.0705
Starting epoch 18 / 500000
t = 3, loss = 0.0723
Starting epoch 19 / 500000
t = 3, loss = 0.0799
Starting epoch 20 / 500000
t = 3, loss = 0.0777
Starting epoch 21 / 500000
t = 3, loss = 0.0799
S

Starting epoch 170 / 500000
t = 3, loss = 0.0498
Starting epoch 171 / 500000
t = 3, loss = 0.0421
Starting epoch 172 / 500000
t = 3, loss = 0.0582
Starting epoch 173 / 500000
t = 3, loss = 0.0539
Starting epoch 174 / 500000
t = 3, loss = 0.0534
Starting epoch 175 / 500000
t = 3, loss = 0.0925
Starting epoch 176 / 500000
t = 3, loss = 0.0437
Starting epoch 177 / 500000
t = 3, loss = 0.0653
Starting epoch 178 / 500000
t = 3, loss = 0.0483
Starting epoch 179 / 500000
t = 3, loss = 0.0443
Starting epoch 180 / 500000
t = 3, loss = 0.0475
Starting epoch 181 / 500000
t = 3, loss = 0.0495
Starting epoch 182 / 500000
t = 3, loss = 0.0501
Starting epoch 183 / 500000
t = 3, loss = 0.0401
Starting epoch 184 / 500000
t = 3, loss = 0.0451
Starting epoch 185 / 500000
t = 3, loss = 0.0497
Starting epoch 186 / 500000
t = 3, loss = 0.0540
Starting epoch 187 / 500000
t = 3, loss = 0.0467
Starting epoch 188 / 500000
t = 3, loss = 0.0457
Starting epoch 189 / 500000
t = 3, loss = 0.0426
Starting epoch 190 /

Starting epoch 337 / 500000
t = 3, loss = 0.0417
Starting epoch 338 / 500000
t = 3, loss = 0.0403
Starting epoch 339 / 500000
t = 3, loss = 0.0418
Starting epoch 340 / 500000
t = 3, loss = 0.0363
Starting epoch 341 / 500000
t = 3, loss = 0.0483
Starting epoch 342 / 500000
t = 3, loss = 0.0438
Starting epoch 343 / 500000
t = 3, loss = 0.0501
Starting epoch 344 / 500000
t = 3, loss = 0.0437
Starting epoch 345 / 500000
t = 3, loss = 0.0493
Starting epoch 346 / 500000
t = 3, loss = 0.0639
Starting epoch 347 / 500000
t = 3, loss = 0.0673
Starting epoch 348 / 500000
t = 3, loss = 0.0475
Starting epoch 349 / 500000
t = 3, loss = 0.0563
Starting epoch 350 / 500000
t = 3, loss = 0.0381
Starting epoch 351 / 500000
t = 3, loss = 0.0407
Starting epoch 352 / 500000
t = 3, loss = 0.0335
Starting epoch 353 / 500000
t = 3, loss = 0.0452
Starting epoch 354 / 500000
t = 3, loss = 0.0418
Starting epoch 355 / 500000
t = 3, loss = 0.0670
Starting epoch 356 / 500000
t = 3, loss = 0.0504
Starting epoch 357 /

Starting epoch 504 / 500000
t = 3, loss = 0.0379
Starting epoch 505 / 500000
t = 3, loss = 0.0394
Starting epoch 506 / 500000
t = 3, loss = 0.0363
Starting epoch 507 / 500000
t = 3, loss = 0.0435
Starting epoch 508 / 500000
t = 3, loss = 0.0531
Starting epoch 509 / 500000
t = 3, loss = 0.0548
Starting epoch 510 / 500000
t = 3, loss = 0.0383
Starting epoch 511 / 500000
t = 3, loss = 0.0372
Starting epoch 512 / 500000
t = 3, loss = 0.0380
Starting epoch 513 / 500000
t = 3, loss = 0.0371
Starting epoch 514 / 500000
t = 3, loss = 0.0565
Starting epoch 515 / 500000
t = 3, loss = 0.0353
Starting epoch 516 / 500000
t = 3, loss = 0.0334
Starting epoch 517 / 500000
t = 3, loss = 0.0446
Starting epoch 518 / 500000
t = 3, loss = 0.0363
Starting epoch 519 / 500000
t = 3, loss = 0.0380
Starting epoch 520 / 500000
t = 3, loss = 0.0405
Starting epoch 521 / 500000
t = 3, loss = 0.0606
Starting epoch 522 / 500000
t = 3, loss = 0.0580
Starting epoch 523 / 500000
t = 3, loss = 0.0363
Starting epoch 524 /

Starting epoch 671 / 500000
t = 3, loss = 0.0379
Starting epoch 672 / 500000
t = 3, loss = 0.0450
Starting epoch 673 / 500000
t = 3, loss = 0.0566
Starting epoch 674 / 500000
t = 3, loss = 0.0403
Starting epoch 675 / 500000
t = 3, loss = 0.0435
Starting epoch 676 / 500000
t = 3, loss = 0.0358
Starting epoch 677 / 500000
t = 3, loss = 0.0367
Starting epoch 678 / 500000
t = 3, loss = 0.0414
Starting epoch 679 / 500000
t = 3, loss = 0.0378
Starting epoch 680 / 500000
t = 3, loss = 0.0371
Starting epoch 681 / 500000
t = 3, loss = 0.0377
Starting epoch 682 / 500000
t = 3, loss = 0.0429
Starting epoch 683 / 500000
t = 3, loss = 0.0362
Starting epoch 684 / 500000
t = 3, loss = 0.0421
Starting epoch 685 / 500000
t = 3, loss = 0.0422
Starting epoch 686 / 500000
t = 3, loss = 0.0392
Starting epoch 687 / 500000
t = 3, loss = 0.0359
Starting epoch 688 / 500000
t = 3, loss = 0.0559
Starting epoch 689 / 500000
t = 3, loss = 0.0507
Starting epoch 690 / 500000
t = 3, loss = 0.0430
Starting epoch 691 /

...

Starting epoch 15774 / 500000
t = 3, loss = 0.0269
Starting epoch 15775 / 500000
t = 3, loss = 0.0301
Starting epoch 15776 / 500000
t = 3, loss = 0.0339
Starting epoch 15777 / 500000
t = 3, loss = 0.0332
Starting epoch 15778 / 500000
t = 3, loss = 0.0310
Starting epoch 15779 / 500000
t = 3, loss = 0.0322
Starting epoch 15780 / 500000
t = 3, loss = 0.0294
Starting epoch 15781 / 500000
t = 3, loss = 0.0299
Starting epoch 15782 / 500000
t = 3, loss = 0.0304
Starting epoch 15783 / 500000
t = 3, loss = 0.0328
Starting epoch 15784 / 500000
t = 3, loss = 0.0318
Starting epoch 15785 / 500000
t = 3, loss = 0.0295
Starting epoch 15786 / 500000
t = 3, loss = 0.0265
Starting epoch 15787 / 500000
t = 3, loss = 0.0285
Starting epoch 15788 / 500000
t = 3, loss = 0.0268
Starting epoch 15789 / 500000
t = 3, loss = 0.0267
Starting epoch 15790 / 500000
t = 3, loss = 0.0269
Starting epoch 15791 / 500000
t = 3, loss = 0.0333
Starting epoch 15792 / 500000
t = 3, loss = 0.0310
Starting epoch 15793 / 500000
t

t = 3, loss = 0.0257
Starting epoch 15935 / 500000
t = 3, loss = 0.0263
Starting epoch 15936 / 500000
t = 3, loss = 0.0288
Starting epoch 15937 / 500000
t = 3, loss = 0.0299
Starting epoch 15938 / 500000
t = 3, loss = 0.0281
Starting epoch 15939 / 500000
t = 3, loss = 0.0277
Starting epoch 15940 / 500000
t = 3, loss = 0.0307
Starting epoch 15941 / 500000
t = 3, loss = 0.0302
Starting epoch 15942 / 500000
t = 3, loss = 0.0269
Starting epoch 15943 / 500000
t = 3, loss = 0.0287
Starting epoch 15944 / 500000
t = 3, loss = 0.0296
Starting epoch 15945 / 500000
t = 3, loss = 0.0321
Starting epoch 15946 / 500000
t = 3, loss = 0.0277
Starting epoch 15947 / 500000
t = 3, loss = 0.0252
Starting epoch 15948 / 500000
t = 3, loss = 0.0279
Starting epoch 15949 / 500000
t = 3, loss = 0.0293
Starting epoch 15950 / 500000
t = 3, loss = 0.0305
Starting epoch 15951 / 500000
t = 3, loss = 0.0291
Starting epoch 15952 / 500000
t = 3, loss = 0.0312
Starting epoch 15953 / 500000
t = 3, loss = 0.0287
Starting e

Starting epoch 16095 / 500000
t = 3, loss = 0.0292
Starting epoch 16096 / 500000
t = 3, loss = 0.0264
Starting epoch 16097 / 500000
t = 3, loss = 0.0274
Starting epoch 16098 / 500000
t = 3, loss = 0.0331
Starting epoch 16099 / 500000
t = 3, loss = 0.0299
Starting epoch 16100 / 500000
t = 3, loss = 0.0295
Starting epoch 16101 / 500000
t = 3, loss = 0.0275
Starting epoch 16102 / 500000
t = 3, loss = 0.0310
Starting epoch 16103 / 500000
t = 3, loss = 0.0326
Starting epoch 16104 / 500000
t = 3, loss = 0.0239
Starting epoch 16105 / 500000
t = 3, loss = 0.0302
Starting epoch 16106 / 500000
t = 3, loss = 0.0285
Starting epoch 16107 / 500000
t = 3, loss = 0.0294
Starting epoch 16108 / 500000
t = 3, loss = 0.0296
Starting epoch 16109 / 500000
t = 3, loss = 0.0272
Starting epoch 16110 / 500000
t = 3, loss = 0.0303
Starting epoch 16111 / 500000
t = 3, loss = 0.0312
Starting epoch 16112 / 500000
t = 3, loss = 0.0323
Starting epoch 16113 / 500000
t = 3, loss = 0.0314
Starting epoch 16114 / 500000
t

t = 3, loss = 0.0282
Starting epoch 16256 / 500000
t = 3, loss = 0.0306
Starting epoch 16257 / 500000
t = 3, loss = 0.0341
Starting epoch 16258 / 500000
t = 3, loss = 0.0297
Starting epoch 16259 / 500000
t = 3, loss = 0.0266
Starting epoch 16260 / 500000
t = 3, loss = 0.0258
Starting epoch 16261 / 500000
t = 3, loss = 0.0293
Starting epoch 16262 / 500000
t = 3, loss = 0.0276
Starting epoch 16263 / 500000
t = 3, loss = 0.0309
Starting epoch 16264 / 500000
t = 3, loss = 0.0270
Starting epoch 16265 / 500000
t = 3, loss = 0.0272
Starting epoch 16266 / 500000
t = 3, loss = 0.0284
Starting epoch 16267 / 500000
t = 3, loss = 0.0285
Starting epoch 16268 / 500000
t = 3, loss = 0.0288
Starting epoch 16269 / 500000
t = 3, loss = 0.0276
Starting epoch 16270 / 500000
t = 3, loss = 0.0287
Starting epoch 16271 / 500000
t = 3, loss = 0.0294
Starting epoch 16272 / 500000
t = 3, loss = 0.0307
Starting epoch 16273 / 500000
t = 3, loss = 0.0264
Starting epoch 16274 / 500000
t = 3, loss = 0.0265
Starting e

Starting epoch 16416 / 500000
t = 3, loss = 0.0280
Starting epoch 16417 / 500000
t = 3, loss = 0.0271
Starting epoch 16418 / 500000
t = 3, loss = 0.0302
Starting epoch 16419 / 500000
t = 3, loss = 0.0286
Starting epoch 16420 / 500000
t = 3, loss = 0.0372
Starting epoch 16421 / 500000
t = 3, loss = 0.0290
Starting epoch 16422 / 500000
t = 3, loss = 0.0342
Starting epoch 16423 / 500000
t = 3, loss = 0.0274
Starting epoch 16424 / 500000
t = 3, loss = 0.0287
Starting epoch 16425 / 500000
t = 3, loss = 0.0273
Starting epoch 16426 / 500000
t = 3, loss = 0.0322
Starting epoch 16427 / 500000
t = 3, loss = 0.0251
Starting epoch 16428 / 500000
t = 3, loss = 0.0284
Starting epoch 16429 / 500000
t = 3, loss = 0.0264
Starting epoch 16430 / 500000
t = 3, loss = 0.0297
Starting epoch 16431 / 500000
t = 3, loss = 0.0278
Starting epoch 16432 / 500000
t = 3, loss = 0.0313
Starting epoch 16433 / 500000
t = 3, loss = 0.0305
Starting epoch 16434 / 500000
t = 3, loss = 0.0333
Starting epoch 16435 / 500000
t

### Load from a previous training

In [ ]:
filename = 'SFD_B75000'

In [ ]:
Denoiser, optimizer, loss = Load_modelloss(filename)
loss = nn.L1Loss().to(device)

In [ ]:
def trainserver(model, loader_train, loss_fn, optimizer, scheduler, num_epochs=1, save_every=1, loss_every=10, filename='denoiser'):

    loss_history = []
    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        model.train()

        for t, (x, y) in enumerate(loader_train):
            x_var = Variable(x.to(device))
            y_var = Variable(y.to(device))

            frame = model(x_var)
            loss = loss_fn(frame, y_var)

            if (t + 1) % loss_every == 0:
                print('t = %d, loss = %.4f' % (t + 1, loss.data))
                loss_history.append(loss)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if (epoch+1) % save_every == 0:
            Save_modelloss(model, optimizer, loss_history,
                           filename+'%s ' % int(epoch+1))

        scheduler.step(loss.data)

In [ ]:
trainserver(Denoiser, trainloader, loss, optimizer, scheduler,
            num_epochs=500000, save_every=25000, loss_every=3, filename=filename)

# Test denoiser and save images

In [ ]:
Data_test = T.Compose([
    T.ToTensor()])

testset = ImageFolderRAM('./CBSD68', 0.1, Data_test,
                         Randomnoise=False, loadram=False, loader=RGB_loader)

testloader = torch.utils.data.DataLoader(testset, shuffle=False, batch_size=1)

In [ ]:
filename1 = 'SFD_25SFD_C_NB75000'
filename1 = 'SFD_25SFD_C_NB75000'
Denoiser, optimizer, loss = Load_modelloss(filename1)
Denoiser2, optimizer, loss2 = Load_modelloss(filename2)



In [ ]:
def Show_result(Denoiser, loader, pause, check=False):
    """
Fonction permettant de montrer les resultats de debuitages d'un reseau de neuronnes
Input :
    - model : reseau à tester
    - loader_train : l'ensemble d'image issu du dataloader
    - pause : temps de pause entre chaque image
    - check : if True cela affiche la moyenne de l'image, le min , le max

Affiche :
    -L'image normale
    -L'image bruitée
    -L'image debruité
    -Les PSNR
"""
    Denoiser.eval()

    plt.ion()
    plt.show()

    # Calcul de la PSNR moyenne
    PSNRmoy = 0
    Compteur = 0

    for i, data in enumerate(loader):
        images, groundtrue = data

        plt.figure(1)
        plt.clf()
        plt.imshow(np.clip(np.transpose(tu.make_grid(
            groundtrue, range=(0, 1), nrow=4).numpy(), (1, 2, 0)), 0, 1))
        plt.axis('off')

        print("Image bruitée")
        plt.figure(2)
        plt.clf()
        plt.imshow(np.clip(np.transpose(tu.make_grid(
            images, range=(0, 1), nrow=4).numpy(), (1, 2, 0)), 0, 1))
        plt.axis('off')

        A = psnr(groundtrue.numpy(), images.numpy(), check=check)
        plt.title(r'%f ' % A)

        print("Image débruitée")
        plt.figure(3)
        plt.clf()
        images = Denoiser(
            Variable(images, requires_grad=False).type(gpu_dtype)).data.cpu()
        plt.imshow(np.clip(np.transpose(tu.make_grid(
            images, range=(0, 1), nrow=4).numpy(), (1, 2, 0)), 0, 1))
        plt.axis('off')

        A = psnr(groundtrue.numpy(), images.numpy(), check=check)
        PSNRmoy += A
        Compteur += 1
        plt.title(r'%f ' % A)

        plt.pause(pause)

    print('PSNR moyen = %f ' % (PSNRmoy/Compteur))

In [ ]:
def Show_result2(Denoiser1, Denoiser2, loader, pause, check=False):
    """
Fonction permettant de comparer les resultats de debuitages d'un reseau de neuronnes
Input :
    - Denoiser1 : debruiteur 1 à tester
    - Denoiser2 : debruiteur 2 à tester
    - loader_train : l'ensemble d'image issu du dataloader
    - pause : temps de pause entre chaque image
    - check : if True cela affiche la moyenne de l'image, le min , le max

Affiche :
    - L'image normale
    - L'image bruitée
    - L'image debruité par 1
    - L'image debruité par 2
    - Les PSNR
"""
    Denoiser1.eval()
    Denoiser2.eval()

    plt.ion()
    plt.show()

    # Calcul de la PSNR moyenne
    PSNRmoy1 = 0
    PSNRmoy2 = 0
    Compteur = 0

    for i, data in enumerate(loader):
        images, groundtrue = data

        plt.figure(1)
        plt.clf()
        plt.imshow(np.clip(np.transpose(tu.make_grid(
            groundtrue, range=(0, 1), nrow=4).numpy(), (1, 2, 0)), 0, 1))
        plt.axis('off')

        print("Image bruitée")
        plt.figure(2)
        plt.clf()
        plt.imshow(np.clip(np.transpose(tu.make_grid(
            images, range=(0, 1), nrow=4).numpy(), (1, 2, 0)), 0, 1))
        plt.axis('off')

        A = psnr(groundtrue.numpy(), images.numpy(), check=check)
        plt.title(r'%f ' % A)

        print("Image débruitée par 1")
        plt.figure(3)
        plt.clf()
        images1 = Denoiser1(
            Variable(images, requires_grad=False).type(gpu_dtype)).data.cpu()
        plt.imshow(np.clip(np.transpose(tu.make_grid(
            images1, range=(0, 1), nrow=4).numpy(), (1, 2, 0)), 0, 1))
        plt.axis('off')

        A = psnr(groundtrue.numpy(), images1.numpy(), check=check)
        PSNRmoy1 += A
        Compteur += 1
        plt.title(r' Denoiseur1 PSNR=%f ' % A)

        print("Image débruitée par 2")
        plt.figure(4)
        plt.clf()
        images2 = Denoiser2(
            Variable(images, requires_grad=False).type(gpu_dtype)).data.cpu()
        plt.imshow(np.clip(np.transpose(tu.make_grid(
            images2, range=(0, 1), nrow=4).numpy(), (1, 2, 0)), 0, 1))
        plt.axis('off')

        A = psnr(groundtrue.numpy(), images2.numpy())
        PSNRmoy2 += A
        plt.title(r' Denoiseur2 PSNR= %f ' % A)

        plt.pause(pause)

    print(' Mean PSNR denoiseur1 = %f ' % (PSNRmoy1/Compteur))
    print(' Mean PSNR denoiseur2 = %f ' % (PSNRmoy2/Compteur))

In [ ]:
# For only one denoiser
Show_result(Denoiser, testloader, 0.1, check=False)

In [ ]:
# To compare 2de
Show_result2(Denoiser, Denoiser2, testloader, 0.1, check=False)

In [ ]:
# Loss Plot

##
plt.show()
plt.ion()

##
plt.figure(5)
plt.clf()
plt.semilogy(range(0, len(loss2))[:10000:10], loss2[:10000:10])

plt.figure(6)
plt.clf()
plt.plot(range(0, len(loss))[:10000:10], loss[:10000:10])

##
plt.figure(6)
plt.clf()
plt.plot(range(0, len(loss2))[1::10], loss2[10000::10])


##
plt.figure(7)
plt.clf()
plt.plot(range(0, len(loss3)), loss3)